In [41]:
# ! pip install pandas numpy scikit-learn plotly matplotlib
# ! pip install nbformat
# ! pip install --upgrade nbformat
# ! pip install tensorflow[and-cuda]

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime as dt
import glob

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from tensorflow.keras.layers import Dense, Input, Dropout, Rescaling, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, Recall

from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import warnings 
warnings.filterwarnings('ignore')

2024-10-07 10:45:43.963833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-07 10:45:43.988208: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-07 10:45:43.995158: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-07 10:45:44.134294: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-07 10:45:45.542531: W tensorflow/compiler/tf2

In [2]:
# Check if GPUs are available for training 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1728290757.278664    6137 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728290757.728890    6137 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728290757.729098    6137 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [44]:
BATCH_SIZE=32

#### Train/test sets

In [45]:
""" # Train set : CIFAKE/train
train_set = tf.keras.utils.image_dataset_from_directory(
    'CIFAKE/train', 
    seed=42,
    image_size=(32,32),
    batch_size=BATCH_SIZE
)

# Test set : CIFAKE > test
validation_set = tf.keras.utils.image_dataset_from_directory(
    'CIFAKE/test', 
    seed=42,
    image_size=(32,32),
    batch_size=BATCH_SIZE
) """

" # Train set : CIFAKE/train\ntrain_set = tf.keras.utils.image_dataset_from_directory(\n    'CIFAKE/train', \n    seed=42,\n    image_size=(32,32),\n    batch_size=BATCH_SIZE\n)\n\n# Test set : CIFAKE > test\nvalidation_set = tf.keras.utils.image_dataset_from_directory(\n    'CIFAKE/test', \n    seed=42,\n    image_size=(32,32),\n    batch_size=BATCH_SIZE\n) "

In [5]:
from PIL import Image

def to_imgArrayCSV(df, set, class_):
    path = f'CIFAKE/{set}/{class_}'
    imgs = glob.glob(f'{path}/*')

    for i, img_path in enumerate(imgs):
        df.loc[len(df.index)] = [set, class_, img_path.split('/')[-1], img_path]
        # df.loc[len(df.index)] = [set, class_, img_path.split('/')[-1], img_np, img_np.shape   ]

def create_CSV():
    df = pd.DataFrame(columns=['set', 'class', 'img_name', 'image_path'])
    # df = pd.DataFrame(columns=['set', 'class', 'img_name', 'np_array', 'np_array.shape'])

    to_imgArrayCSV(df, 'train', 'FAKE')
    to_imgArrayCSV(df, 'train', 'REAL')
    to_imgArrayCSV(df, 'test', 'FAKE')
    to_imgArrayCSV(df, 'test', 'REAL')
    df.to_csv('CIFAKE_imgpath.csv') # obs index

In [6]:
df = pd.read_csv('CIFAKE_imgpath.csv')
df = df[['set', 'class', 'img_name', 'image_path']]
pd.crosstab(df['set'], df['class'])

class,FAKE,REAL
set,,
test,10000,10000
train,50000,50000


In [7]:
df['class'] = df['class'].map({'FAKE':0, 'REAL':1})
df

,set,class,img_name,image_path
0,train,0,5996 (9).jpg,CIFAKE/train/FAKE/5996 (9).jpg
1,train,0,5996.jpg,CIFAKE/train/FAKE/5996.jpg
2,train,0,5997 (10).jpg,CIFAKE/train/FAKE/5997 (10).jpg
3,train,0,5997 (2).jpg,CIFAKE/train/FAKE/5997 (2).jpg
4,train,0,5997 (3).jpg,CIFAKE/train/FAKE/5997 (3).jpg
...,...,...,...,...
119995,test,1,0996 (7).jpg,CIFAKE/test/REAL/0996 (7).jpg
119996,test,1,0996 (8).jpg,CIFAKE/test/REAL/0996 (8).jpg
119997,test,1,0996 (9).jpg,CIFAKE/test/REAL/0996 (9).jpg
119998,test,1,0996.jpg,CIFAKE/test/REAL/0996.jpg


In [8]:
X_train = []
y_train = []

df_train = df[df.set == 'train'].reset_index(drop=True)

for i in range(len(df_train)):
    img_path = df_train.image_path[i]
    img_class = df_train['class'][i]

    img_pil = Image.open(img_path)
    img_np = np.asarray(img_pil)

    X_train.append(img_np)
    y_train.append(img_class)

X_train = np.asarray(X_train, dtype='uint8')
y_train = np.asarray(y_train)

In [9]:
X_test = []
y_test = []

df_test = df[df.set == 'test'].reset_index(drop=True)

for i in range(len(df_test)):
    img_path = df_test.image_path[i]
    img_class = df_test['class'][i]

    img_pil = Image.open(img_path)
    img_np = np.asarray(img_pil)

    X_test.append(img_np)
    y_test.append(img_class)

X_test = np.asarray(X_test, dtype='uint8')
y_test = np.asarray(y_test)

In [10]:
print(f'X_train shape: {X_train.shape} | y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape} | y_test shape: {y_test.shape}')

X_train shape: (100000, 32, 32, 3) | y_train shape: (100000,)
X_test shape: (20000, 32, 32, 3) | y_test shape: (20000,)


#### EDA

In [11]:
import glob
print('Class distribution: ')
print( f'Train REAL images: {len(glob.glob('CIFAKE/train/REAL/*'))}'  )
print( f'Train FAKE images: {len(glob.glob('CIFAKE/train/FAKE/*'))}'  )

print( f'Test REAL images: {len(glob.glob('CIFAKE/test/REAL/*'))}'  )
print( f'Test FAKE images: {len(glob.glob('CIFAKE/test/FAKE/*'))}'  )

Class distribution: 
Train REAL images: 50000
Train FAKE images: 50000
Test REAL images: 10000
Test FAKE images: 10000


In [53]:
# Show random image

#### Image normalisation

In [12]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

ValueError: Found array with dim 4. StandardScaler expected <= 2.

In [54]:
# Image normalisation

#### Build model

In [13]:
# Build model
def CNN_model01():
    model = Sequential([
        Rescaling(1./255),
        Conv2D(32, 3, activation='relu'),
        MaxPooling2D(),
        Flatten(),

        Dense(24, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])

    model.compile( optimizer=Adam(learning_rate=0.01), 
                  loss=BinaryCrossentropy(),
                  metrics = ['accuracy', Precision(), Recall()],                
                  )

    model.build(input_shape=(None, 32, 32, 3))
    return model

In [84]:
# CNN_model01().summary()

#### Train model

In [14]:
N_EPOCHS = 30
# BATCH SIZE ?? # NOTE

In [26]:
from sklearn.model_selection import GridSearchCV
from datetime import datetime as dt


def train(model):
  time_start = dt.now()
  print(f'Start training, time: {time_start.time()}')

  history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=N_EPOCHS,
    verbose=1
  )

  print(f'Time elapsed: {dt.now() - time_start}')

  return history

# Train with GRID search
def train_gridSearch(model):
  param_grid = dict(
      epochs=[25],
      batch_size=[32],
  )

  grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=4, cv=10, scoring='accuracy')

  start_t = dt.now()
  print(f'GridSearch start time: {start_t.hour}:{start_t.minute:02}:{start_t.second:02}')
  grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test))

  print(f'GridSearch complete. Time elapsed: {dt.now()-start_t}')

  # Best results 
  print(f'Best parameters: {grid_result.best_params_}')
  print(f'Best result (R2): {grid_result.best_score_}')

  history = grid_result.best_estimator_.history_

  return history

In [27]:
from scikeras.wrappers import KerasClassifier

# model = CNN_model01()
model = KerasClassifier(model=CNN_model01())


history = train_gridSearch(model)

GridSearch start time: 10:55:48


2024-10-07 10:55:49.622230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-07 10:55:49.622422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-07 10:55:49.638251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-07 10:55:49.640018: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-07 10:55:49.643676: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/mnt/sde1/My_Files/Code/Git_Repos/hve_projects/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/mnt/sde1/My_Files/Code/Git_Repos/hve_projects/.venv/lib/python3.12/site-packages/scikeras/wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/mnt/sde1/My_Files/Code/Git_Repos/hve_projects/.venv/lib/python3.12/site-packages/scikeras/wrappers.py", line 770, in fit
    self._fit(
  File "/mnt/sde1/My_Files/Code/Git_Repos/hve_projects/.venv/lib/python3.12/site-packages/scikera

In [23]:
from sklearn.model_selection import cross_val_score

model = KerasClassifier(model=CNN_model01())

fit_params

k_fold_acc = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy', fit_params=fit_params)

2024-10-07 10:52:39.824408: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 276480000 exceeds 10% of free system memory.
2024-10-07 10:52:40.094500: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 276480000 exceeds 10% of free system memory.
I0000 00:00:1728291160.889062    8175 service.cc:146] XLA service 0x7628c0008710 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728291160.889079    8175 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 6GB, Compute Capability 6.1
2024-10-07 10:52:40.936681: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-07 10:52:41.105217: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 156/2813 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5670 - loss: 0.6989 - precision_5: 0.5672 - recall_5: 0.4310

I0000 00:00:1728291162.518580    8175 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2813/2813 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.7166 - loss: 0.5530 - precision_5: 0.6969 - recall_5: 0.7571


2024-10-07 10:52:46.537570: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30720000 exceeds 10% of free system memory.
2024-10-07 10:52:46.568163: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30720000 exceeds 10% of free system memory.


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


2024-10-07 10:52:47.649457: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 276480000 exceeds 10% of free system memory.


2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.4988 - loss: 0.6994 - precision_5: 0.4995 - recall_5: 0.5349
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7106 - loss: 0.5596 - precision_5: 0.6920 - recall_5: 0.7613
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.5018 - loss: 0.6986 - precision_5: 0.4996 - recall_5: 0.4303
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7328 - loss: 0.5436 - precision_5: 0.7117 - recall_5: 0.7779
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7177 - loss: 0.5555 - precision_5: 0.7002 - recall_5: 0.7606
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7207 - loss: 0.5562 - precision_5: 0.7058 - recall_5: 0.7597
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 

In [25]:
k_fold_acc

array([0.7985, 0.5   , 0.7955, 0.5   , 0.8096, 0.7749, 0.7969, 0.793 ,
       0.5   , 0.8143])

#### Visualisation

In [24]:
cols = pd.DataFrame(history.history).columns
cols

NameError: name 'history' is not defined

In [65]:
hist = history.history

n = 5
train_col = cols[2]
val_col = f'val_{train_col}'

fig = make_subplots(rows=2, cols=2, subplot_titles=("Loss over epochs", f"{train_col} over epochs"), vertical_spacing=0.07)

# Loss
fig.add_trace( go.Scatter(x=list(range(len(hist['loss']))), y=hist['loss'], mode='lines+markers', name='Training Loss'), row=1, col=1 )
fig.add_trace( go.Scatter(x=list(range(len(hist['val_loss']))), y=hist['val_loss'], mode='lines+markers', name='Validation Loss'), row=1, col=1 )

# 
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=1, col=2 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=1, col=2)

#
train_col = cols[3]
val_col = f'val_{train_col}'
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=2, col=1 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=2, col=1)

train_col = 'accuracy'
val_col = f'val_{train_col}'
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=2, col=2 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=2, col=2)


fig.update_xaxes(title_text="Epochs", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=1)
fig.update_yaxes(title_text=f"{train_col}", row=1, col=2)
fig.update_yaxes(title_text=f"{train_col}", row=2, col=1)
fig.update_yaxes(title_text=f"{train_col}", row=2, col=2)

fig.update_layout(
    # title_text="Training and validation metrics over epochs",
    showlegend=True,
    margin=dict(l=10, r=10, b=10, t=30),
    width=1400, height=800
)

for annotation in fig['layout']['annotations']:
    annotation['y'] = annotation['y'] + 0.002

fig

In [66]:
# Confusion matrix
# conf_matrix = confusion_matrix(y_test, y_pred_classes)

#### Class activation map